In [7]:
import geopandas as gpd

import emaremes as mrms
import dataretrieval.nwis as nwis
from folium import Map

mrms.fetch.path_config.add_path("./hackatton_data", make_prefered=True)

Prefered path to store *new* Gribfiles is  hackatton_data


In [2]:
cook_county = gpd.read_file(
    "/home/edwin/Desktop/GIS_drafts/Municipalities/Cook_County_Border.geojson"
)

files = mrms.fetch.timerange("2025-04-02", "2025-04-07", frequency="30min", data_type="precip_rate")
df = mrms.ts.polygon.query_files(files, cook_county)

-> 241 files were requested...


In [3]:
site_info, md = nwis.get_info(
    countyCd=["17031"],  # Cook County, IL
    parameterCd=["00045"],  # Instantaneous Precipitation
    startDt="2023-01-01",
    endDt="2025-04-07",
)

In [4]:
site_info.head()

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no,geometry
0,USGS,05530990,"SALT CREEK AT ROLLING MEADOWS, IL",ST,420337.41,880059.97,42.060392,-88.016658,N,5,...,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-88.01666 42.06039)
1,USGS,05536290,"LITTLE CALUMET RIVER AT SOUTH HOLLAND, IL",ST,413625.30,873551.30,41.607028,-87.597583,X,F,...,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.59758 41.60703)
2,USGS,413104087440001,"RAIN GAGE AT MATTESON, IL",AT,413104.00,874400.00,41.517778,-87.733333,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAWS0,POINT (-87.73333 41.51778)
3,USGS,413113087342201,"RAIN GAGE NEAR CHICAGO HEIGHTS, IL",AT,413115.00,873525.00,41.520868,-87.590321,M,S,...,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,00100,POINT (-87.59032 41.52087)
4,USGS,413115087352501,"RAIN GAGE AT DEER CREEK NEAR CHICAGO HEIGHTS, IL",AT,413115.00,873525.00,41.520833,-87.590278,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAWS00,POINT (-87.59028 41.52083)


In [5]:
center = float(cook_county.centroid.y), float(cook_county.centroid.x)

/tmp/ipykernel_63989/4002738056.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = float(cook_county.centroid.y), float(cook_county.centroid.x)
/tmp/ipykernel_63989/4002738056.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  center = float(cook_county.centroid.y), float(cook_county.centroid.x)


In [8]:
m = Map(zoom_start=10, location=center)
cook_county.explore(m=m, color="green")
site_info.explore(marker_kwds={"radius": 10}, m=m, color="black")
display(m)

In [ ]:
# bBox: string or list of strings A contiguous range of decimal latitude and longitude, starting with the
# west longitude, then the south latitude, then the east longitude, and
# then the north latitude

# get daily values (dv)
df_daily = nwis.get_record(service="dv", sites=site_info["site_no"], start="", end=str_end)